In [ ]:
## use RNA-STAR conda environment
from pathlib import Path
import traceback
import argparse
import pandas as pd
import pysam

### Loading in data

In [18]:
left = pd.read_csv("UNUAR_motif_sites_mRNA.tsv", sep = "\t")
left["Motif"] = left["Motif"].str.replace("U", "T") ## each BAM file has sequences w/ Thymine (T) instead of Uracil (U)
right = pd.read_excel("SupplementaryTable1.xlsx")

df = pd.merge(left, right, how = "left", on = "Motif")
genome_coord = df[["Chrom", "GenomicModBase"]].to_numpy() ## faster processing

### Opening BAM file

In [16]:
df.head(1)

,TranscriptID,Motif,Region,Chrom,Strand,TranscriptPosStart,TranscriptPosEnd,TranscriptModBase,GenomicModBase,TranscriptLength,DistFromAUG,DistFromSTOP,DistFromExonStart,DistFromExonEnd,fit_A,fit_B,fit_R
0,rna-NM_001134855.2-2,TCTAG,CDS,NW_018654708.1,-,1358,1363,1360,373,1733,880,151,101,372,0.716983,0.051299,0.678453


In [ ]:
# If UNUAR site is at a position where there are 0 reads (btween r1 end, r2 start), then impute n/a into base counts. After all data is appended to TSV, drop all rows with n/a
# Append counts to a dictionary, append to dataframe in chunks, then append that dataframe to tsv in chunk
# Write (create) columns in tsv before appending dataframe
# Ensure that total count == A + G + C + T + Deletions
# Add error handling at the end

In [ ]:
## my code
def pysam_pileup(bamfile, chrom, mod_base, base_counts):
    for pileupcolumn in bamfile.pileup(str(chrom), int(mod_base-1), int(mod_base), truncate = True): ## pysam is 0-based, while GenomicModBase was 1-based
        pileupcolumn.set_min_base_quality(0) ## prevent pysam from filtering based on base quality
        if pileupcolumn.pos != int(mod_base-1): ## if not at exact pos, skip to next iteration
            continue
        for pileupread in pileupcolumn.pileups:
            if pileupread.is_del and not pileupread.is_refskip:
                base_counts["del"] += 1
            elif not pileupread.is_refskip:
                base = pileupread.alignment.query_sequence[pileupread.query_position] ## taken from example in manual; returns base letter
                base_counts[str(base)] += 1

def count_base(input_bam_name, fafile):
    bamfile = pysam.AlignmentFile(input_bam_name, "rb")
    results = []
    """
    chrom: value in "Chrom" column (e.g., NW_018654708.1)
    mod_base: value in "GenomicModBase" column (e.g., 373)
    """
    for row in genome_coord:
        chrom = row[0]
        mod_base = row[1]
        base_counts = {"A": 0, "C": 0, "G": 0, "T": 0, "del": 0}

        pysam_pileup(chrom, mod_base, base_counts)
        bamfile.close()

        results.append({"Chrom": chrom,
                        "GenomicModBase": mod_base,
                        **base_counts}) ## unpack base_counts dict in this dict

In [ ]:
def open_bam(folder_name, fasta_dir):
    current_path = Path.cwd()
    input_dir = current_path/"realignments"/folder_name
    fafile = pysam.FastaFile(fasta_dir)
    
    try: 
        for subfolder in input_dir.iterdir():
            if subfolder.is_dir():
                processed_folder = input_dir/f"{subfolder.name}_realigned"
                
                for bam in subfolder.glob("*.bam"):
                    input_bam_name = Path(bam) ## turn string from list back into filepath

                    count_base(input_bam_name, fasta_dir)

    except Exception as e:
        print(f"Failed to calculate deletion rates in {folder_name}: {e}")
        traceback.print_exc()
        raise

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description = "Calculates observed and real deletion rates for every read in a BAM file.")
    parser.add_argument("--folder_name", help = "Name of processed_fastqs folder", required = True)
    parser.add_argument("--fasta_dir", help = "Path of reference fasta file", required = True)
    args = parser.parse_args()

    print("Starting realignment...")
    open_bam(args.folder_name, args.fasta_dir)
    print("Realignment finished.")

### Test code

In [ ]:
# chunk_size = 100000
# for i in range(0, len(results), chunk_size):
#     chunk = results[i:i+chunk_size]
#     df_chunk = pd.DataFrame(chunk)
#     df_chunk.to_csv("output.tsv", sep="\t", index=False,
#                     mode="a", header=(i == 0))